In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/all_universe_predictions.csv')

In [ ]:
# 1. Mark each prediction as ambiguous
#df['is_ambiguous'] = df['pred_set'].apply(lambda labels: 1 if len(labels) > 1 else 0)

In [5]:
from ast import literal_eval

df['pred_set'] = df['pred_set'].apply(
    lambda x: literal_eval(x) if isinstance(x, str) else x
)

In [6]:
df['is_ambiguous'] = df['pred_set'].apply(lambda preds: 1 if len(preds) > 1 else 0)

In [7]:
df.head

<bound method NDFrame.head of         pred_set  true_label  frau1  nongerman  nongerman_male  \
0            {0}           0      1        0.0               0   
1            {0}           0      0        0.0               0   
2            {0}           0      1        1.0               0   
3            {0}           0      0        0.0               0   
4            {0}           0      1        0.0               0   
...          ...         ...    ...        ...             ...   
1043791      {0}           0      0        0.0               0   
1043792   {0, 1}           0      0        0.0               0   
1043793      {0}           0      0        0.0               0   
1043794      {0}           0      1        0.0               0   
1043795      {0}           1      1        0.0               0   

         nongerman_female   model        feature_set threshold_policy  \
0                       0  logreg     with_protected            top15   
1                       0  logr

In [ ]:
# 2. Compute the fraction of ambiguous predictions for each combination of (universe decisions + subgroup)
#group_cols = ['feature_set', 'model', 'threshold_policy', 'gender', 'nationality']
#fractions = df.groupby(group_cols)['is_ambiguous'].mean().reset_index()

In [ ]:
# `fractions` now contains one row per (feature_set, model, threshold_policy, gender, nationality) 
# with the mean of `is_ambiguous`, which is the fraction of predictions that are ambiguous in that subgroup.
#print(fractions.head())  # For debugging: view the first few rows of fraction results

# Melt the subgroup flags into one column so you can group by them

In [8]:
# List of binary subgroup columns
subgroup_cols = ['frau1', 'nongerman', 'nongerman_male', 'nongerman_female']

# Melt to long format: one row per (row, subgroup) where value == 1
df_long = df.melt(
    id_vars=['feature_set', 'model', 'threshold_policy', 'is_ambiguous'],
    value_vars=subgroup_cols,
    var_name='subgroup',
    value_name='is_member'
)

# Filter to keep only the subgroup memberships (i.e., rows where the person is in that group)
df_long = df_long[df_long['is_member'] == 1]


# Compute the fraction of ambiguous predictions per (universe, subgroup)

In [9]:
group_cols = ['feature_set', 'model', 'threshold_policy', 'subgroup']
fractions = df_long.groupby(group_cols)['is_ambiguous'].mean().reset_index()
fractions.rename(columns={'is_ambiguous': 'ambiguity_fraction'}, inplace=True)


# For each universe, compute the maximum pairwise difference in ambiguity fraction between subgroups

In [10]:
# Compute disparity per universe = max - min ambiguity_fraction across subgroups
disparity_df = fractions.groupby(['feature_set', 'model', 'threshold_policy'])['ambiguity_fraction']\
                        .agg(lambda x: x.max() - x.min())\
                        .reset_index(name='ambiguity_disparity')
